In [1]:
import torch
import os
from torch.utils.cpp_extension import load

os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [2]:
%load_ext wurlitzer

### Simple addition Kernel

In [3]:
addition_module = load(
    name="add",
    sources=["csrc/add.cu"], 
    extra_cuda_cflags=["-O2", "-Xcompiler", "-Werror", "-Xcompiler", "-Wall"], 
    verbose=True
)

Using /home/ganesh/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ganesh/.cache/torch_extensions/py310_cu121/add/build.ninja...
Building extension module add...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/2] /usr/local/cuda/bin/nvcc --generate-dependencies-with-compile --dependency-output add.cuda.o.d -DTORCH_EXTENSION_NAME=add -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/ganesh/.local/share/virtualenvs/cudamode-lectures-SlOoH9rC/lib/python3.10/site-packages/torch/include -isystem /home/ganesh/.local/share/virtualenvs/cudamode-lectures-SlOoH9rC/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -isystem /home/ganesh/.local/share/virtualenvs/cudamode-lectures-SlOoH9rC/lib/python3.10/site-packages/torch/include/TH -isystem /home/ganesh/.local/share/virtualenvs/cudamode-lectures-SlOoH9rC/lib/python3.10/site-packages/torch/include/THC -isystem /usr/local/cuda/include -isystem /home/ganesh/.pyenv/versions/3.10.13/include/python3.10 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_BFLOAT16_CONVERSIONS__ -D__CUDA_NO_HALF2_

Loading extension module add...


In [30]:
gen = torch.Generator(device='cuda')
gen.manual_seed(42)

a = torch.randn(size=(1000000, 10), dtype=torch.float32, device='cuda:0', generator=gen).contiguous()
b = torch.randn(size=(1000000, 10), dtype=torch.float32, device='cuda:0', generator=gen).contiguous()

In [31]:
%time

c = a + b

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 8.11 µs


In [32]:
%time 

c_cuda = addition_module.add(a, b)

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 8.34 µs
n: 10000000


In [28]:
c_cuda.shape

torch.Size([1000000, 10])

In [29]:
torch.testing.assert_close(c, c_cuda)